### **MonthlyMean snowCover in Zugspitze/Germany**

In [1]:
#Import earth engine
import ee

#Trigger the authentication flow
ee.Authenticate()

#Initialize the library
ee.Initialize(project='modis-410817')

In [2]:
# Import geemap
import geemap as emap

In [3]:
# Define the region of interest (ROI)
#roi = ee.Geometry.Rectangle(10.97, 47.40, 11.47, 47.50)
roi = ee.Geometry.Rectangle([10.956, 47.381, 11.213, 47.470])

# Load MODIS Terra Snow Cover Daily Global 500m
image_collection = ee.ImageCollection("MODIS/061/MOD10A1").select('NDSI_Snow_Cover')

# Filter the image collection for the specified date ranges and region of interest
filtered_collection = (image_collection
                      .filter(ee.Filter.calendarRange(2000, 2020, 'year'))
                      .filter(ee.Filter.Or(
                          ee.Filter.calendarRange(1, 3, 'month'),
                          ee.Filter.calendarRange(10, 12, 'month')
                      ))
                      .filterBounds(roi))

# Define the list of months and years
#months = [1, 2, 3, 10, 11, 12]   ## To get data from 2001 to 2020
months = [ 10, 11, 12]   ## To get data in 2000
#months = [ 1, 2, 3]   ## To get data in 2020

#years = list(range(2001, 2020))   ## To get data from 2001 to 2020
years = list(range(2000, 2001))   ## To get data in 2000
#years = list(range(2020, 2021))   ## To get data in 2020

# Create a list of images by mapping over years and months
image_list = ee.List([
    (filtered_collection
     .filter(ee.Filter.calendarRange(y, y, 'year'))
     .filter(ee.Filter.calendarRange(m, m, 'month'))
     .filterMetadata('system:time_start', 'not_equals', None)  # Filter out images without time_start property
     .mean()
     .set('month', m)
     .set('year', y)
     .set('system:time_start', filtered_collection.first().get('system:time_start'))  # Add time_start property to mean image
    )
    for y in years for m in months
])

# Create an ImageCollection from the list of images
by_month_year = ee.ImageCollection.fromImages(image_list)

In [4]:
# Get the image IDs
image_ids = by_month_year.aggregate_array('system:index').getInfo()
print('Total images', len(image_ids))

Total images 3


In [5]:
#Visualization parameters
snow_cover_vis = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['black', '0dffff', '0524ff', 'ffffff']
}

In [6]:
# Get a list of images
image_list = by_month_year.toList(by_month_year.size())

#Clip for displaying in map
img = ee.Image(image_list.get(0)).clip(roi)

In [7]:
# Calculate the area of the state
state_area = img.geometry().area()

# Convert the area to square kilometers and round the result
state_area_sq_km = ee.Number(state_area).divide(1e6)

# Print the result
print(state_area_sq_km.getInfo())

191.33409159257155


In [8]:
#Create a center the map object
Map = emap.Map()
Map.center_object(roi, zoom=12)
# Add the layers to the map object.
Map.addLayer(img,snow_cover_vis,'Snow cover')

# Draw the rectangle on the map
Map.addLayer(roi, {'color': 'red'}, 'roi')

#Add a layer control panel to the map
Map.addLayerControl()

#Display the map
Map

Map(center=[47.42555925039793, 11.084500000000109], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Loop through the images
for i in range(image_list.length().getInfo()):
    img = ee.Image(image_list.get(i))

    # Cast all bands to Int16
    img = img.toUint16()

    # Extract month and year information from image properties
    month = img.get('month')
    year = img.get('year')

    # Define the export parameters
    export_params = {
        'image': img.select('NDSI_Snow_Cover'),
        'description': 'Snow_Cover_Zugspitze_{}_{}_{}'.format(year.getInfo(), month.getInfo(), i + 1),
        'fileNamePrefix': '{}_{}'.format(year.getInfo(), month.getInfo()),  # Include year and month in the file name prefix
        'folder': 'Snow_Cover_Zugspitze1',
        'scale': 500,
        'region': roi.getInfo()['coordinates'],
        'crs': 'EPSG:4326',
        'maxPixels': 1e10
    }

    # Start the export task
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    print('Started task: ', i + 1)

Started task:  1
Started task:  2
Started task:  3
